### Calculating distance between existing residential locations and commercial locations

Process:
- For each residential building, check the distance to each commerical building 
- If the distance is less than a mile, create a binary variable that takes the value 1, 0 otherwise

In [1]:
# Import libraries
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import haversine as hs


In [2]:
# Get the residential and commercial building location data
buildings_df = gpd.read_file('../processed_data/relevant_buildings.shp')
buildings_df


,CLASS,class_reco,hood,geoid10,tractce10,geometry
0,C,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-80.01230 40.38309, -80.01255 40.383..."
1,R,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-80.01382 40.38638, -80.01380 40.386..."
2,R,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-80.01309 40.38253, -80.01307 40.382..."
3,R,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-80.01399 40.38554, -80.01399 40.385..."
4,R,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-80.00736 40.38204, -80.00729 40.381..."
...,...,...,...,...,...,...
116273,C,commercial,Point Breeze,420039811001,981100,"POLYGON ((-79.90934 40.44247, -79.90928 40.442..."
116274,R,2-Unit Residential,Squirrel Hill South,420039803001,980300,"POLYGON ((-79.91447 40.43197, -79.91443 40.431..."
116275,C,commercial,Squirrel Hill South,420039803001,980300,"POLYGON ((-79.90925 40.42686, -79.90910 40.426..."
116276,R,1-Unit Residential,Squirrel Hill South,420039803001,980300,"POLYGON ((-79.91190 40.43303, -79.91190 40.433..."


In [3]:
# Remove grocery store buildings
buildings_df = buildings_df[buildings_df['class_reco'] != 'Grocery Store']
buildings_df

,CLASS,class_reco,hood,geoid10,tractce10,geometry
0,C,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-80.01230 40.38309, -80.01255 40.383..."
1,R,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-80.01382 40.38638, -80.01380 40.386..."
2,R,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-80.01309 40.38253, -80.01307 40.382..."
3,R,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-80.01399 40.38554, -80.01399 40.385..."
4,R,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-80.00736 40.38204, -80.00729 40.381..."
...,...,...,...,...,...,...
116273,C,commercial,Point Breeze,420039811001,981100,"POLYGON ((-79.90934 40.44247, -79.90928 40.442..."
116274,R,2-Unit Residential,Squirrel Hill South,420039803001,980300,"POLYGON ((-79.91447 40.43197, -79.91443 40.431..."
116275,C,commercial,Squirrel Hill South,420039803001,980300,"POLYGON ((-79.90925 40.42686, -79.90910 40.426..."
116276,R,1-Unit Residential,Squirrel Hill South,420039803001,980300,"POLYGON ((-79.91190 40.43303, -79.91190 40.433..."


In [4]:
# Do the CRS converstion (converting polygons to centroid)
df_points = buildings_df.copy()

# Project to CRS
df_points['geometry_crs'] = df_points['geometry'].to_crs(epsg=3035)

# Convert to centroids
df_points['centroids'] = df_points['geometry_crs'].centroid.to_crs(4326)
df_points

,CLASS,class_reco,hood,geoid10,tractce10,geometry,geometry_crs,centroids
0,C,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-80.01230 40.38309, -80.01255 40.383...","POLYGON ((-1280260.228 6136598.189, -1280260.0...",POINT (-80.01241 40.38317)
1,R,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-80.01382 40.38638, -80.01380 40.386...","POLYGON ((-1279946.994 6136891.874, -1279945.5...",POINT (-80.01375 40.38637)
2,R,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-80.01309 40.38253, -80.01307 40.382...","POLYGON ((-1280329.396 6136636.826, -1280318.8...",POINT (-80.01300 40.38258)
3,R,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-80.01399 40.38554, -80.01399 40.385...","POLYGON ((-1280035.495 6136864.334, -1280039.0...",POINT (-80.01401 40.38548)
4,R,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-80.00736 40.38204, -80.00729 40.381...","POLYGON ((-1280292.699 6136126.260, -1280296.6...",POINT (-80.00738 40.38199)
...,...,...,...,...,...,...,...,...
116273,C,commercial,Point Breeze,420039811001,981100,"POLYGON ((-79.90934 40.44247, -79.90928 40.442...","POLYGON ((-1272618.173 6130848.222, -1272613.6...",POINT (-79.90929 40.44247)
116274,R,2-Unit Residential,Squirrel Hill South,420039803001,980300,"POLYGON ((-79.91447 40.43197, -79.91443 40.431...","POLYGON ((-1273769.610 6130756.263, -1273768.9...",POINT (-79.91442 40.43189)
116275,C,commercial,Squirrel Hill South,420039803001,980300,"POLYGON ((-79.90925 40.42686, -79.90910 40.426...","POLYGON ((-1274210.890 6130057.717, -1274201.8...",POINT (-79.90929 40.42676)
116276,R,1-Unit Residential,Squirrel Hill South,420039803001,980300,"POLYGON ((-79.91190 40.43303, -79.91190 40.433...","POLYGON ((-1273621.056 6130591.820, -1273610.6...",POINT (-79.91181 40.43308)


In [6]:
# Check matrix size for distance calculations
print(f"# of commercial buildings = {len(df_points[df_points['class_reco'] == 'commercial'])}")
print(f"# of residential buildings = {len(df_points) - len(df_points[df_points['class_reco'] == 'commercial'])}")

matrix_size = len(df_points[df_points['class_reco'] == 'commercial']) * len(df_points) - len(df_points[df_points['class_reco'] == 'commercial'])
print(matrix_size)


# of commercial buildings = 6895
# of residential buildings = 109324
801323110


In [7]:
# Converting point objects to lat long

def convert_point_to_latlong(point):
    lat = point.x
    long = point.y

    return (lat, long)

# apply the function
df_points['coordinates'] = df_points.apply(lambda row: convert_point_to_latlong(row['centroids']), axis = 1)

df_points



,CLASS,class_reco,hood,geoid10,tractce10,geometry,geometry_crs,centroids,coordinates
0,C,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-80.01230 40.38309, -80.01255 40.383...","POLYGON ((-1280260.228 6136598.189, -1280260.0...",POINT (-80.01241 40.38317),"(-80.01241198145034, 40.383174075305014)"
1,R,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-80.01382 40.38638, -80.01380 40.386...","POLYGON ((-1279946.994 6136891.874, -1279945.5...",POINT (-80.01375 40.38637),"(-80.01375293703208, 40.38637193448172)"
2,R,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-80.01309 40.38253, -80.01307 40.382...","POLYGON ((-1280329.396 6136636.826, -1280318.8...",POINT (-80.01300 40.38258),"(-80.01299982129609, 40.382580778371924)"
3,R,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-80.01399 40.38554, -80.01399 40.385...","POLYGON ((-1280035.495 6136864.334, -1280039.0...",POINT (-80.01401 40.38548),"(-80.0140061375439, 40.385478011132015)"
4,R,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-80.00736 40.38204, -80.00729 40.381...","POLYGON ((-1280292.699 6136126.260, -1280296.6...",POINT (-80.00738 40.38199),"(-80.00738482859445, 40.38199146881134)"
...,...,...,...,...,...,...,...,...,...
116273,C,commercial,Point Breeze,420039811001,981100,"POLYGON ((-79.90934 40.44247, -79.90928 40.442...","POLYGON ((-1272618.173 6130848.222, -1272613.6...",POINT (-79.90929 40.44247),"(-79.90928821722198, 40.442466005506255)"
116274,R,2-Unit Residential,Squirrel Hill South,420039803001,980300,"POLYGON ((-79.91447 40.43197, -79.91443 40.431...","POLYGON ((-1273769.610 6130756.263, -1273768.9...",POINT (-79.91442 40.43189),"(-79.91442414504114, 40.431892538017415)"
116275,C,commercial,Squirrel Hill South,420039803001,980300,"POLYGON ((-79.90925 40.42686, -79.90910 40.426...","POLYGON ((-1274210.890 6130057.717, -1274201.8...",POINT (-79.90929 40.42676),"(-79.90929182071689, 40.42676068310648)"
116276,R,1-Unit Residential,Squirrel Hill South,420039803001,980300,"POLYGON ((-79.91190 40.43303, -79.91190 40.433...","POLYGON ((-1273621.056 6130591.820, -1273610.6...",POINT (-79.91181 40.43308),"(-79.91181037643665, 40.43308374924944)"


In [14]:
df_points= df_points[['geoid10','tractce10']]
df_points

,geoid10,tractce10
0,420031918003,191800
1,420031918003,191800
2,420031918003,191800
3,420031918003,191800
4,420031918003,191800
...,...,...
116273,420039811001,981100
116274,420039803001,980300
116275,420039803001,980300
116276,420039803001,980300


In [15]:
df_points.drop_duplicates()

,geoid10,tractce10
0,420031918003,191800
2815,420035628002,562800
3563,420035630002,563000
4808,420032023004,202300
6345,420032904003,290400
...,...,...
116121,420039822001,982200
116135,420030404001,040400
116214,420039805001,980500
116219,420039811001,981100


In [8]:
# Test time (shorter dataset)
df_points_small = df_points.sample(n=1000)

df_points_small_res = df_points_small[df_points_small['class_reco'] != 'commercial']
df_points_small_res = df_points_small_res[['geoid10', 'tractce10', 'coordinates']]
df_points_small_res.rename(columns={"coordinates": "res_coordinates", "geoid10":"geoid_res", "tractce10":"tract_id_res"}, inplace=True)


df_points_small_comm = df_points_small[df_points_small['class_reco'] == 'commercial']
df_points_small_comm = df_points_small_comm[['geoid10', 'tractce10', 'coordinates']]
df_points_small_comm.rename(columns={"coordinates": "comm_coordinates", "geoid10":"geoid_comm", "tractce10":"tract_id_comm"}, inplace=True)



In [10]:
df_points_small_comm

,geoid_comm,tract_id_comm,comm_coordinates
102511,420031115004,111500,"(-79.91888971597386, 40.458675087808764)"
94998,420035623005,562300,"(-79.94308454988142, 40.40735140048198)"
43306,420031014004,101400,"(-79.92032745098739, 40.48769070879683)"
107625,420031017002,101700,"(-79.94043021561649, 40.46499635719381)"
39743,420030901002,090100,"(-79.96519663182936, 40.474335449755685)"
9639,420031920002,192000,"(-80.01933318439869, 40.405587511335725)"
60366,420034810002,481000,"(-79.98725747414414, 40.41034939952864)"
83745,420035626002,562600,"(-80.04134348877834, 40.4437028631385)"
72227,420030203001,020300,"(-79.9848850975365, 40.44971612820952)"
12363,420031919001,191900,"(-80.01813054006267, 40.39268436203251)"


In [9]:
df_points_small_res

,geoid_res,tract_id_res,res_coordinates
113978,420031208002,120800,"(-79.91584708967055, 40.46254634129956)"
112941,420030903002,090300,"(-79.95423923898308, 40.463823349340686)"
34060,420032615001,261500,"(-80.01051168554302, 40.47524787807522)"
84119,420032507001,250700,"(-80.01766676599543, 40.458613625211974)"
64812,420031916003,191600,"(-80.01614378148398, 40.4130271864654)"
...,...,...,...
65218,420031916003,191600,"(-80.02758676572678, 40.41650342699534)"
111258,420030806002,080600,"(-79.94140172036518, 40.461421720339075)"
29770,420032607002,260700,"(-80.01209072796313, 40.48205109503867)"
94442,420031403004,140300,"(-79.92411267125628, 40.44268507839717)"


In [11]:
# Cross join the 2 files
df_points_small_comm['key'] = 1
df_points_small_res['key'] = 1
  
df_cross_joined = pd.merge(df_points_small_comm, df_points_small_res, on ='key').drop("key", 1)

df_cross_joined


C:\Users\ABHILA~1\AppData\Local\Temp/ipykernel_16500/1011094173.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_cross_joined = pd.merge(df_points_small_comm, df_points_small_res, on ='key').drop("key", 1)


,geoid_comm,tract_id_comm,comm_coordinates,geoid_res,tract_id_res,res_coordinates
0,420031115004,111500,"(-79.91888971597386, 40.458675087808764)",420031208002,120800,"(-79.91584708967055, 40.46254634129956)"
1,420031115004,111500,"(-79.91888971597386, 40.458675087808764)",420030903002,090300,"(-79.95423923898308, 40.463823349340686)"
2,420031115004,111500,"(-79.91888971597386, 40.458675087808764)",420032615001,261500,"(-80.01051168554302, 40.47524787807522)"
3,420031115004,111500,"(-79.91888971597386, 40.458675087808764)",420032507001,250700,"(-80.01766676599543, 40.458613625211974)"
4,420031115004,111500,"(-79.91888971597386, 40.458675087808764)",420031916003,191600,"(-80.01614378148398, 40.4130271864654)"
...,...,...,...,...,...,...
55514,420030103002,010300,"(-79.9933378623689, 40.4387787998778)",420031916003,191600,"(-80.02758676572678, 40.41650342699534)"
55515,420030103002,010300,"(-79.9933378623689, 40.4387787998778)",420030806002,080600,"(-79.94140172036518, 40.461421720339075)"
55516,420030103002,010300,"(-79.9933378623689, 40.4387787998778)",420032607002,260700,"(-80.01209072796313, 40.48205109503867)"
55517,420030103002,010300,"(-79.9933378623689, 40.4387787998778)",420031403004,140300,"(-79.92411267125628, 40.44268507839717)"


In [12]:
# Calculate haversine distance
df_cross_joined['distance'] = df_cross_joined.apply(lambda row: hs.haversine(row['comm_coordinates'], row['res_coordinates'], unit=hs.Unit.MILES), axis = 1)
df_cross_joined


,geoid_comm,tract_id_comm,comm_coordinates,geoid_res,tract_id_res,res_coordinates,distance
0,420031115004,111500,"(-79.91888971597386, 40.458675087808764)",420031208002,120800,"(-79.91584708967055, 40.46254634129956)",0.215378
1,420031115004,111500,"(-79.91888971597386, 40.458675087808764)",420030903002,090300,"(-79.95423923898308, 40.463823349340686)",2.443210
2,420031115004,111500,"(-79.91888971597386, 40.458675087808764)",420032615001,261500,"(-80.01051168554302, 40.47524787807522)",6.333619
3,420031115004,111500,"(-79.91888971597386, 40.458675087808764)",420032507001,250700,"(-80.01766676599543, 40.458613625211974)",6.824844
4,420031115004,111500,"(-79.91888971597386, 40.458675087808764)",420031916003,191600,"(-80.01614378148398, 40.4130271864654)",6.742041
...,...,...,...,...,...,...,...
55514,420030103002,010300,"(-79.9933378623689, 40.4387787998778)",420031916003,191600,"(-80.02758676572678, 40.41650342699534)",2.381387
55515,420030103002,010300,"(-79.9933378623689, 40.4387787998778)",420030806002,080600,"(-79.94140172036518, 40.461421720339075)",3.598781
55516,420030103002,010300,"(-79.9933378623689, 40.4387787998778)",420032607002,260700,"(-80.01209072796313, 40.48205109503867)",1.395793
55517,420030103002,010300,"(-79.9933378623689, 40.4387787998778)",420031403004,140300,"(-79.92411267125628, 40.44268507839717)",4.783237


In [22]:
# Calculate distance using geopandas distance method to check time
# Test time (shorter dataset)
df_points_small = df_points.sample(n=20000)

df_points_small_res = df_points_small[df_points_small['class_reco'] != 'commercial']
df_points_small_res = df_points_small_res[['geoid10', 'tractce10', 'centroids']]
df_points_small_res.rename(columns={"centroids": "res_centroids", "geoid10":"geoid_res", "tractce10":"tract_id_res"}, inplace=True)


df_points_small_comm = df_points_small[df_points_small['class_reco'] == 'commercial']
df_points_small_comm = df_points_small_comm[['geoid10', 'tractce10', 'centroids']]
df_points_small_comm.rename(columns={"centroids": "comm_centroids", "geoid10":"geoid_comm", "tractce10":"tract_id_comm"}, inplace=True)

# Cross join the 2 files
df_points_small_comm['key'] = 1
df_points_small_res['key'] = 1
  
df_cross_joined = pd.merge(df_points_small_comm, df_points_small_res, on ='key').drop("key", 1)

# Calcuate distance
df_cross_joined['distance_prelim'] = df_cross_joined.apply(lambda row: row['res_centroids'].distance(row['comm_centroids']), axis = 1)

## Longer time

C:\Users\ABHILA~1\AppData\Local\Temp/ipykernel_27564/2674570165.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_cross_joined = pd.merge(df_points_small_comm, df_points_small_res, on ='key').drop("key", 1)


In [ ]:
# Do the full dataset based on Haversine distance (NOT RUN YET)

df_points_res = df_points[df_points['class_reco'] != 'commercial']
df_points_res = df_points_res[['geoid10', 'tractce10', 'coordinates']]
df_points_res.rename(columns={"coordinates": "res_coordinates", "geoid10":"geoid_res", "tractce10":"tract_id_res"}, inplace=True)


df_points_comm = df_points[df_points['class_reco'] == 'commercial']
df_points_comm = df_points_comm[['geoid10', 'tractce10', 'coordinates']]
df_points_comm.rename(columns={"coordinates": "comm_coordinates", "geoid10":"geoid_comm", "tractce10":"tract_id_comm"}, inplace=True)

# Cross join the 2 files
df_points_comm['key'] = 1
df_points_res['key'] = 1
  
df_cross_joined = pd.merge(df_points_comm, df_points_res, on ='key').drop("key", 1)
 
df_cross_joined['distance'] = df_cross_joined.apply(lambda row: hs.haversine(row['comm_coordinates'], row['res_coordinates'], unit=hs.Unit.MILES), axis = 1)
df_cross_joined




In [3]:
# Testing helper function
import helper_distance_calculation as hdc




In [4]:
df_points_small = buildings_df.sample(n=1000)
df_points_small

,CLASS,class_reco,hood,geoid10,tractce10,geometry
57972,R,1-Unit Residential,South Side Slopes,420031706001,170600,"POLYGON ((-79.98093 40.42440, -79.98086 40.424..."
57673,R,1-Unit Residential,South Side Slopes,420031706001,170600,"POLYGON ((-79.98531 40.42039, -79.98527 40.420..."
1087,R,1-Unit Residential,Brookline,420031918003,191800,"POLYGON ((-80.02331 40.39227, -80.02318 40.392..."
54882,C,commercial,Knoxville,420033001005,300100,"POLYGON ((-79.99030 40.41135, -79.99005 40.411..."
63904,R,1-Unit Residential,Beechview,420031916003,191600,"POLYGON ((-80.02813 40.40947, -80.02801 40.409..."
...,...,...,...,...,...,...
100342,R,1-Unit Residential,Squirrel Hill North,420031401003,140100,"POLYGON ((-79.92936 40.43835, -79.92925 40.438..."
43060,R,1-Unit Residential,Morningside,420031014004,101400,"POLYGON ((-79.92853 40.48196, -79.92846 40.481..."
47579,R,1-Unit Residential,Upper Lawrenceville,420031011003,101100,"POLYGON ((-79.95372 40.48048, -79.95372 40.480..."
65946,R,1-Unit Residential,Beltzhoover,420035624005,562400,"POLYGON ((-80.00183 40.41013, -80.00184 40.410..."


In [12]:
distance_res_comm = hdc.calculate_access(
                            geopandas_dataframe=df_points_small,
                            building_type_1='Residential',
                            building_type_2='commercial',
                            identifier_column='class_reco', 
                            geo_column='geometry', 
                            output_format='dataframe'
)

distance_res_comm

c:\Users\Abhilash Biswas\anaconda3\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Abhilash Biswas\anaconda3\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Abhilash Biswas\anaconda3\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

,geoid_Residential,tract_id_Residential,Residential_coordinates,geoid_commercial,tract_id_commercial,commercial_coordinates,distance
0,420031706001,170600,"(-79.98090155031424, 40.424296554292354)",420033001005,300100,"(-79.99018755306798, 40.411286601765205)",0.660369
1,420031706001,170600,"(-79.98090155031424, 40.424296554292354)",420032507001,250700,"(-80.01727465954725, 40.45625518561571)",2.542231
2,420031706001,170600,"(-79.98090155031424, 40.424296554292354)",420035626002,562600,"(-80.03204658950153, 40.43410659008704)",3.535743
3,420031706001,170600,"(-79.98090155031424, 40.424296554292354)",420035632002,563200,"(-80.00139252643818, 40.44906366989132)",1.446693
4,420031706001,170600,"(-79.98090155031424, 40.424296554292354)",420031702001,170200,"(-79.98559070814896, 40.42927133991059)",0.329460
...,...,...,...,...,...,...,...
58089,420031113004,111300,"(-79.91776033070686, 40.46781857275382)",420039807001,980700,"(-80.00301174986558, 40.43280719019389)",5.905389
58090,420031113004,111300,"(-79.91776033070686, 40.46781857275382)",420030901002,090100,"(-79.95800676742283, 40.47594501598761)",2.782494
58091,420031113004,111300,"(-79.91776033070686, 40.46781857275382)",420031115004,111500,"(-79.92065298030234, 40.46154713367803)",0.213771
58092,420031113004,111300,"(-79.91776033070686, 40.46781857275382)",420031115004,111500,"(-79.93014839444871, 40.463944172297424)",0.857214


In [13]:
distance_res_groc = hdc.calculate_access(
                            geopandas_dataframe=df_points_small,
                            building_type_1='Residential',
                            building_type_2='Grocery',
                            identifier_column='class_reco', 
                            geo_column='geometry', 
                            output_format='dataframe'
)

distance_res_groc

c:\Users\Abhilash Biswas\anaconda3\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Abhilash Biswas\anaconda3\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Abhilash Biswas\anaconda3\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

,geoid_Residential,tract_id_Residential,Residential_coordinates,geoid_Grocery,tract_id_Grocery,Grocery_coordinates,distance
0,420031706001,170600,"(-79.98090155031424, 40.424296554292354)",420031408003,140800,"(-79.9225728859554, 40.43566540928973)",4.032457
1,420031706001,170600,"(-79.9852292105387, 40.42037390248725)",420031408003,140800,"(-79.9225728859554, 40.43566540928973)",4.333061
2,420031918003,191800,"(-80.02320219082878, 40.39229998274667)",420031408003,140800,"(-79.9225728859554, 40.43566540928973)",6.972367
3,420031916003,191600,"(-80.02807936022836, 40.40941120539879)",420031408003,140800,"(-79.9225728859554, 40.43566540928973)",7.296639
4,420031911002,191100,"(-80.02821679072031, 40.438256092473104)",420031408003,140800,"(-79.9225728859554, 40.43566540928973)",7.299365
...,...,...,...,...,...,...,...
932,420031401003,140100,"(-79.9292925411029, 40.438257082578644)",420031408003,140800,"(-79.9225728859554, 40.43566540928973)",0.465339
933,420031014004,101400,"(-79.92850156209069, 40.48193648457734)",420031408003,140800,"(-79.9225728859554, 40.43566540928973)",0.693223
934,420031011003,101100,"(-79.9536573915891, 40.4804768331484)",420031408003,140800,"(-79.9225728859554, 40.43566540928973)",2.214809
935,420035624005,562400,"(-80.00190198621334, 40.41006648020965)",420031408003,140800,"(-79.9225728859554, 40.43566540928973)",5.489781
